In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [3]:
import subprocess
import shlex
import json

def findVideoMetada(pathToInputVideo):
    cmd = "ffprobe -v quiet -print_format json -show_streams"
    args = shlex.split(cmd)
    args.append(pathToInputVideo)
    ffprobeOutput = subprocess.check_output(args).decode('utf-8')
    ffprobeOutput = json.loads(ffprobeOutput)

#     import pprint
#     pp = pprint.PrettyPrinter(indent=2)
#     pp.pprint(ffprobeOutput)

    # for example, find height and width
#     height = ffprobeOutput['streams'][0]['height']
#     width = ffprobeOutput['streams'][0]['width']
    return ffprobeOutput['streams']

In [4]:
findVideoMetada('../data/original/000.mp4')

[{'index': 0,
  'codec_name': 'h264',
  'codec_long_name': 'H.264 / AVC / MPEG-4 AVC / MPEG-4 part 10',
  'profile': 'High 4:4:4 Predictive',
  'codec_type': 'video',
  'codec_time_base': '1/50',
  'codec_tag_string': 'avc1',
  'codec_tag': '0x31637661',
  'width': 640,
  'height': 480,
  'coded_width': 640,
  'coded_height': 480,
  'has_b_frames': 2,
  'pix_fmt': 'yuv444p',
  'level': 30,
  'chroma_location': 'left',
  'refs': 1,
  'is_avc': 'true',
  'nal_length_size': '4',
  'r_frame_rate': '25/1',
  'avg_frame_rate': '25/1',
  'time_base': '1/12800',
  'start_pts': 0,
  'start_time': '0.000000',
  'duration_ts': 202752,
  'duration': '15.840000',
  'bit_rate': '402660',
  'bits_per_raw_sample': '8',
  'nb_frames': '396',
  'disposition': {'default': 1,
   'dub': 0,
   'original': 0,
   'comment': 0,
   'lyrics': 0,
   'karaoke': 0,
   'forced': 0,
   'hearing_impaired': 0,
   'visual_impaired': 0,
   'clean_effects': 0,
   'attached_pic': 0,
   'timed_thumbnails': 0},
  'tags': {'l

In [11]:
def create_compressed_dataset(source_path, target_path):
    for i in tqdm(os.scandir(source_path)):
        path = i.path.replace('\\', '/')
        metadata = findVideoMetada(path)
        assert len(metadata) in [1, 2], f'{metadata}'
        old_bitrate = int(metadata[0]['bit_rate'])
        bitrate_percentages = [0.77, 0.6]
        crfs = [23, 40]
        new_bitrates = [int(old_bitrate*perc) for perc in bitrate_percentages]
        
        if not os.path.exists(f'{target_path}'):
            os.mkdir(f'{target_path}')
        
        for perc, bitrate, crf in zip(bitrate_percentages, new_bitrates, crfs):
            if not os.path.exists(f'{target_path}/{perc}'):
                os.mkdir(f'{target_path}/{perc}')
            cmd = f'ffmpeg -y -i {path} -c:v libx264 -crf {crf} {target_path}/{perc}/{i.name}'
            args = shlex.split(cmd)
            ffprobeOutput = subprocess.check_output(args).decode('utf-8')
            
create_compressed_dataset('../data/original', '../data/output')

1001it [3:04:34, 11.58s/it]


In [18]:
import os
for i in os.scandir('../data'):
    print(i.path)

../data\000.mp4
../data\001.mp4
../data\002.mp4
../data\007.mp4
../data\obama_fake.mp4


In [20]:
from pytube import YouTube
YouTube('https://www.youtube.com/watch?v=cQ54GDm1eL0').streams.first().download()

'C:\\Users\\C64062\\Documents\\FaceForensics\\compression_detection\\You Won’t Believe What Obama Says In This Video! 😉.mp4'

In [ ]:
model = DecisionTreeClassifier()
target_path = '../data/'
for p in os.scandir(target_path)